# 회귀분석으로 집값예측하기

1. 데이터 준비

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [4]:
# 데이터 불러오기 (CSV 또는 병합된 데이터셋)
data = pd.read_csv("C:/fintech_service/06_machine_learning/data/real_final_merge_data.csv", encoding='UTF8') 

In [5]:
data.head()

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,date,기준금리,달러환율
0,강원특별자치도 양구군 양구읍 상리,경림,59.400,"5,000",11,1998.0,24,2005-09-23,3.25,1061.83
1,경기도 의정부시 신곡동,신일1,59.878,"5,750",3,1997.0,1151,2005-10-20,3.50,1077.02
2,경기도 의정부시 신곡동,풍림,49.830,"5,500",8,1998.0,1151,2005-10-20,3.50,1077.02
3,경기도 의정부시 민락동,한라비발디,84.990,"12,750",19,2003.0,1151,2005-10-30,3.50,1061.83
4,경기도 의정부시 용현동,용현현대1차,129.730,"12,200",11,1992.0,1151,2005-11-21,3.50,1060.19


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041451 entries, 0 to 10041450
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   시군구       object 
 1   단지명       object 
 2   전용면적(㎡)   float64
 3   거래금액(만원)  object 
 4   층         int64  
 5   건축년도      float64
 6   학원수       int64  
 7   date      object 
 8   기준금리      float64
 9   달러환율      float64
dtypes: float64(4), int64(2), object(4)
memory usage: 766.1+ MB


In [13]:
# '단지명' 컬럼의 유니크 값 확인
unique_danji_names = data['단지명'].unique()

# 유니크 값 출력
print(unique_danji_names)

# 유니크 값의 개수 확인
print(f"'단지명' 유니크 값의 개수: {len(unique_danji_names)}")

['경림' '신일1' '풍림' ... '북구청역푸르지오에듀포레' '서대구영무예다음아파트' '검산이지움라프라임아파트']
'단지명' 유니크 값의 개수: 35135


In [14]:
# '단지명' 컬럼에서 동일한 값의 개수 확인
danji_name_counts = data['단지명'].value_counts()

# 결과 출력
print(danji_name_counts)


단지명
현대                 87242
주공                 34370
삼성                 34064
주공2                33718
우성                 32791
                   ...  
한울라온채                  1
국봉빌라                   1
한국서부발전사원아파트새빛마을        1
동명휴티스오션시티아파트           1
검산이지움라프라임아파트           1
Name: count, Length: 35135, dtype: int64


In [15]:
# 1. 거래금액(만원) 변환 (천단위 콤마 제거 후 숫자로 변환)
data['거래금액(만원)'] = data['거래금액(만원)'].str.replace(',', '').astype(float)

# 2. 날짜 컬럼 변환 및 연도와 월 추출
data['date'] = pd.to_datetime(data['date'])
data['연도'] = data['date'].dt.year
data['월'] = data['date'].dt.month

data

,시군구,단지명,전용면적(㎡),거래금액(만원),층,건축년도,학원수,date,기준금리,달러환율,연도,월
0,강원특별자치도 양구군 양구읍 상리,경림,59.4000,5000.0,11,1998.0,24,2005-09-23,3.25,1061.83,2005,9
1,경기도 의정부시 신곡동,신일1,59.8780,5750.0,3,1997.0,1151,2005-10-20,3.50,1077.02,2005,10
2,경기도 의정부시 신곡동,풍림,49.8300,5500.0,8,1998.0,1151,2005-10-20,3.50,1077.02,2005,10
3,경기도 의정부시 민락동,한라비발디,84.9900,12750.0,19,2003.0,1151,2005-10-30,3.50,1061.83,2005,10
4,경기도 의정부시 용현동,용현현대1차,129.7300,12200.0,11,1992.0,1151,2005-11-21,3.50,1060.19,2005,11
...,...,...,...,...,...,...,...,...,...,...,...,...
10041446,충청남도 서산시 대산읍 대산리,한미,58.4700,3000.0,5,1992.0,363,2024-08-31,3.50,1408.72,2024,8
10041447,경상북도 안동시 법흥동,안동강변펠리시아아파트,116.4457,52000.0,7,2017.0,390,2024-08-31,3.50,1408.72,2024,8
10041448,경상북도 구미시 오태동,신한디아체101동102동,59.9800,4700.0,1,1999.0,1275,2024-08-31,3.50,1408.72,2024,8
10041449,경상남도 양산시 동면 석산리,양산이지더원리버포레,78.2990,34200.0,15,2017.0,928,2024-08-31,3.50,1408.72,2024,8


In [16]:
# 3. 필요 없는 컬럼 제거 (단지명, date 등 분석에 불필요한 변수)
data = data.drop(columns=['date'])

In [18]:
data = data.drop(columns=['단지명'])
data

,시군구,전용면적(㎡),거래금액(만원),층,건축년도,학원수,기준금리,달러환율,연도,월
0,강원특별자치도 양구군 양구읍 상리,59.4000,5000.0,11,1998.0,24,3.25,1061.83,2005,9
1,경기도 의정부시 신곡동,59.8780,5750.0,3,1997.0,1151,3.50,1077.02,2005,10
2,경기도 의정부시 신곡동,49.8300,5500.0,8,1998.0,1151,3.50,1077.02,2005,10
3,경기도 의정부시 민락동,84.9900,12750.0,19,2003.0,1151,3.50,1061.83,2005,10
4,경기도 의정부시 용현동,129.7300,12200.0,11,1992.0,1151,3.50,1060.19,2005,11
...,...,...,...,...,...,...,...,...,...,...
10041446,충청남도 서산시 대산읍 대산리,58.4700,3000.0,5,1992.0,363,3.50,1408.72,2024,8
10041447,경상북도 안동시 법흥동,116.4457,52000.0,7,2017.0,390,3.50,1408.72,2024,8
10041448,경상북도 구미시 오태동,59.9800,4700.0,1,1999.0,1275,3.50,1408.72,2024,8
10041449,경상남도 양산시 동면 석산리,78.2990,34200.0,15,2017.0,928,3.50,1408.72,2024,8


In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041451 entries, 0 to 10041450
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   시군구       object 
 1   전용면적(㎡)   float64
 2   거래금액(만원)  float64
 3   층         int64  
 4   건축년도      float64
 5   학원수       int64  
 6   기준금리      float64
 7   달러환율      float64
 8   연도        int32  
 9   월         int32  
dtypes: float64(5), int32(2), int64(2), object(1)
memory usage: 689.5+ MB


In [22]:
data['시군구'].value_counts()

시군구
서울특별시 노원구 상계동           51787
경기도 시흥시 정왕동             41173
경기도 수원영통구 영통동           37615
경기도 군포시 산본동             37166
부산광역시 해운대구 좌동           36323
                        ...  
충청남도 천안시 서북구 성환읍 성환리        1
전라북도 전주덕진구 산정동              1
전북특별자치도 군산시 임피면 읍내리         1
충청북도 청주시 상당구 수동             1
경상북도 경주시 건천읍 화천리            1
Name: count, Length: 4393, dtype: int64

In [23]:
# '시군구' 컬럼을 ' '로 나눠서 리스트에 넣고, 리스트의 길이 구하기
data['시군구_리스트_길이'] = data['시군구'].apply(lambda x: len(str(x).split(' ')))

# 리스트 길이의 유니크값 확인
unique_lengths = data['시군구_리스트_길이'].unique()

unique_lengths


array([4, 3, 2, 5], dtype=int64)

In [24]:
# 리스트 길이의 count 값 확인
length_counts = data['시군구_리스트_길이'].value_counts()

# count 값 출력
print(length_counts)

시군구_리스트_길이
3    8609712
4    1394035
2      29646
5       8058
Name: count, dtype: int64


In [25]:
# '시군구_리스트_길이'가 5인 경우, 맨 마지막 항목을 삭제하고 4자리로 만들기
data.loc[data['시군구_리스트_길이'] == 5, '시군구'] = data.loc[data['시군구_리스트_길이'] == 5, '시군구'].apply(lambda x: ' '.join(str(x).split(' ')[:-1]))

# 변경 후, 다시 리스트 길이 업데이트
data['시군구_리스트_길이'] = data['시군구'].apply(lambda x: len(str(x).split(' ')))

# 결과 확인
print(data['시군구'].head())
print(data['시군구_리스트_길이'].value_counts())


0    강원특별자치도 양구군 양구읍 상리
1          경기도 의정부시 신곡동
2          경기도 의정부시 신곡동
3          경기도 의정부시 민락동
4          경기도 의정부시 용현동
Name: 시군구, dtype: object
시군구_리스트_길이
3    8609712
4    1402093
2      29646
Name: count, dtype: int64


In [26]:
data

,시군구,전용면적(㎡),거래금액(만원),층,건축년도,학원수,기준금리,달러환율,연도,월,시군구_리스트_길이
0,강원특별자치도 양구군 양구읍 상리,59.4000,5000.0,11,1998.0,24,3.25,1061.83,2005,9,4
1,경기도 의정부시 신곡동,59.8780,5750.0,3,1997.0,1151,3.50,1077.02,2005,10,3
2,경기도 의정부시 신곡동,49.8300,5500.0,8,1998.0,1151,3.50,1077.02,2005,10,3
3,경기도 의정부시 민락동,84.9900,12750.0,19,2003.0,1151,3.50,1061.83,2005,10,3
4,경기도 의정부시 용현동,129.7300,12200.0,11,1992.0,1151,3.50,1060.19,2005,11,3
...,...,...,...,...,...,...,...,...,...,...,...
10041446,충청남도 서산시 대산읍 대산리,58.4700,3000.0,5,1992.0,363,3.50,1408.72,2024,8,4
10041447,경상북도 안동시 법흥동,116.4457,52000.0,7,2017.0,390,3.50,1408.72,2024,8,3
10041448,경상북도 구미시 오태동,59.9800,4700.0,1,1999.0,1275,3.50,1408.72,2024,8,3
10041449,경상남도 양산시 동면 석산리,78.2990,34200.0,15,2017.0,928,3.50,1408.72,2024,8,4


In [27]:
data = data.drop(columns=['시군구_리스트_길이'])

In [28]:
data

,시군구,전용면적(㎡),거래금액(만원),층,건축년도,학원수,기준금리,달러환율,연도,월
0,강원특별자치도 양구군 양구읍 상리,59.4000,5000.0,11,1998.0,24,3.25,1061.83,2005,9
1,경기도 의정부시 신곡동,59.8780,5750.0,3,1997.0,1151,3.50,1077.02,2005,10
2,경기도 의정부시 신곡동,49.8300,5500.0,8,1998.0,1151,3.50,1077.02,2005,10
3,경기도 의정부시 민락동,84.9900,12750.0,19,2003.0,1151,3.50,1061.83,2005,10
4,경기도 의정부시 용현동,129.7300,12200.0,11,1992.0,1151,3.50,1060.19,2005,11
...,...,...,...,...,...,...,...,...,...,...
10041446,충청남도 서산시 대산읍 대산리,58.4700,3000.0,5,1992.0,363,3.50,1408.72,2024,8
10041447,경상북도 안동시 법흥동,116.4457,52000.0,7,2017.0,390,3.50,1408.72,2024,8
10041448,경상북도 구미시 오태동,59.9800,4700.0,1,1999.0,1275,3.50,1408.72,2024,8
10041449,경상남도 양산시 동면 석산리,78.2990,34200.0,15,2017.0,928,3.50,1408.72,2024,8


In [29]:
# # 필요한 경우 병합된 데이터를 저장
# data.to_csv("C:/fintech_service/06_machine_learning/data/precision_data.csv", index=False)


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10041451 entries, 0 to 10041450
Data columns (total 10 columns):
 #   Column    Dtype  
---  ------    -----  
 0   시군구       object 
 1   전용면적(㎡)   float64
 2   거래금액(만원)  float64
 3   층         int64  
 4   건축년도      float64
 5   학원수       int64  
 6   기준금리      float64
 7   달러환율      float64
 8   연도        int32  
 9   월         int32  
dtypes: float64(5), int32(2), int64(2), object(1)
memory usage: 689.5+ MB


In [33]:
data['시군구'].value_counts()

시군구
서울특별시 노원구 상계동          51787
경기도 시흥시 정왕동            41173
경기도 수원영통구 영통동          37615
경기도 군포시 산본동            37166
부산광역시 해운대구 좌동          36323
                       ...  
전라북도 전주덕진구 산정동             1
전북특별자치도 부안군 진서면 곰소리        1
강원특별자치도 동해시 나안동            1
전북특별자치도 군산시 임피면 읍내리        1
경상북도 경주시 건천읍 화천리           1
Name: count, Length: 4298, dtype: int64

In [36]:
# # 1. 시군구 컬럼에 원핫 인코딩 적용
# data = pd.get_dummies(data, columns=['시군구'], drop_first=True)

2. 탐색적 데이터 분석 (EDA)

In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# 종속변수와 독립변수 간의 관계 시각화
sns.pairplot(data)
plt.show()

C:\Users\5-06\miniforge3\envs\fintech\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Glyph 51204 (\N{HANGUL SYLLABLE JEON}) missing from font(s) DejaVu Sans.
  self._figure.tight_layout(*args, **kwargs)
C:\Users\5-06\miniforge3\envs\fintech\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Glyph 50857 (\N{HANGUL SYLLABLE YONG}) missing from font(s) DejaVu Sans.
  self._figure.tight_layout(*args, **kwargs)
C:\Users\5-06\miniforge3\envs\fintech\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Glyph 47732 (\N{HANGUL SYLLABLE MYEON}) missing from font(s) DejaVu Sans.
  self._figure.tight_layout(*args, **kwargs)
C:\Users\5-06\miniforge3\envs\fintech\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Glyph 51201 (\N{HANGUL SYLLABLE JEOG}) missing from font(s) DejaVu Sans.
  self._figure.tight_layout(*args, **kwargs)
C:\Users\5-06\miniforge3\envs\fintech\lib\site-packages\seaborn\axisgrid.py:123: UserWarning: Glyph 13217 (\N{SQUARE M SQUARED}) missing from font(s) D

KeyboardInterrupt: 

In [9]:
# 상관관계 히트맵
corr_matrix = data.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

ValueError: could not convert string to float: '강원특별자치도 양구군 양구읍 상리'

3. 데이터 분할

In [ ]:
# 종속변수와 독립변수 설정
X = data.drop(columns=['거래금액(만원)'])
y = data['거래금액(만원)']

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 분할된 데이터 확인
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


4. 회귀모델 선택 및 학습

In [ ]:
from sklearn.linear_model import LinearRegression

# 선형 회귀 모델 생성 및 학습
model = LinearRegression()
model.fit(X_train, y_train)

# 회귀계수 출력
print("회귀계수:", model.coef_)
print("절편:", model.intercept_)


5. 모델 평가

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# 테스트 데이터에 대한 예측
y_pred = model.predict(X_test)

# 모델 성능 평가
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R² Score: {r2}")


6. 모델 개선

In [ ]:
from sklearn.linear_model import Ridge, Lasso

# Ridge 회귀 (L2 정규화)
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_ridge_pred = ridge_model.predict(X_test)

# Lasso 회귀 (L1 정규화)
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)
y_lasso_pred = lasso_model.predict(X_test)

# 평가
ridge_r2 = r2_score(y_test, y_ridge_pred)
lasso_r2 = r2_score(y_test, y_lasso_pred)

print(f"Ridge R² Score: {ridge_r2}")
print(f"Lasso R² Score: {lasso_r2}")


7. 결과 해석 및 시각화

In [ ]:
# 실제 값과 예측 값 시각화
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='실제 값', marker='o')
plt.plot(y_pred, label='예측 값', marker='x')
plt.title("실제 값과 예측 값 비교")
plt.legend()
plt.show()